<a href="https://colab.research.google.com/github/abhigyanbalyan/News-Classifier/blob/main/Minor_1_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from  sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bbc-text.csv to bbc-text (1).csv


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
data=pd.read_csv("bbc-text.csv")
le=LabelEncoder()
data["category_id"]=le.fit_transform(data["category"])

category_id_df = data[['category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'category']].values)

tfidf = TfidfVectorizer(encoding='latin-1', ngram_range=(1, 2), stop_words='english',min_df=5)
features = tfidf.fit_transform(data.text).toarray()
labels = data.category_id

model=LogisticRegression(random_state=0)

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, data.index, test_size=0.35, random_state=0)

model.fit(X_train, y_train)

#y_pred_proba = model.predict_proba(X_test)
#y_pred = model.predict(X_test)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [ ]:
def summarisation(text):
  stopwords = list(STOP_WORDS)
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(text)
  #tokens = [token.text for token in doc]
  

  word_frequencies = {}
  for word in doc:
    if word.text.lower() not in stopwords:
     
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  max_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency
  #Tokenisation of sentences
  sentence_tokens = [sent for sent in doc.sents]
  #Sentence score
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent] += word_frequencies[word.text.lower()]  
  from heapq import nlargest
  select_length = int(len(sentence_tokens)*0.3)
  summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
  final_summary = [word.text for word in summary]
  summary = ' '.join(final_summary)
  print(summary)
  print(len(summary))

In [ ]:
text = [""" Jio 5G service will rollout in India in the second half of 2021, Reliance Industries Chairman and Managing Director
 Mukesh Ambani revealed during his keynote at India Mobile Congress 2020 on Tuesday.
  The Indian billionaire noted that the 5G service offered by Jio would be a “testimony” to the government's Atmanirbhar Bharat (self-reliant India) policy.
   Apart from rolling out 5G in the country, Jio is in development of an affordable Android phone in collaboration with Google that is likely to debut in the coming months.
Ambani, 63, said that the policy steps were needed to accelerate early rollout of 5G in the country."""]
#summarisation(text)
text_features = tfidf.transform(text)
predictions = model.predict(text_features)
for text, predicted in zip(text, predictions):
  print('"{}............"'.format(text[:50]))
  print("  - Predicted as: '{}'".format(id_to_category[predicted]))
  print("")
  print(len(text))

" Jio 5G service will rollout in India in the secon............"
  - Predicted as: 'business'

627


In [ ]:
 summarisation(text)

Ambani, 63, said that the policy steps were needed to accelerate early rollout of 5G in the country.
100
